# ArangoDB NetworkX Adapter Getting Started Guide  

![arangodb](assets/logos/ArangoDB_logo.png) 

In this Notebook we learn how to export Graphs from [ArangoDB](https://www.arangodb.com/), a multi-model Graph Database into [NetworkX](https://networkx.github.io/), the swiss army knife for graph analysis ion python.

# Setup

In [1]:
!pip install adbnx-adapter
!pip install python-arango
!pip install PyYAML==5.1.1
!pip3 install pyarango

Processing /home/admin2/.cache/pip/wheels/16/27/a1/775c62ddea7bfa62324fd1f65847ed31c55dadb6051481ba3f/PyYAML-5.1.1-cp37-cp37m-linux_x86_64.whl
ERROR: mlflow 0.9.0.1 requires Flask, which is not installed.
ERROR: mlflow 0.9.0.1 has requirement cloudpickle==0.6.1, but you'll have cloudpickle 1.2.2 which is incompatible.
  Found existing installation: PyYAML 5.2
ERROR: Cannot uninstall 'PyYAML'. It is a distutils installed project and thus we cannot accurately determine which files belong to it which would lead to only a partial uninstall.
Traceback (most recent call last):
  File "/home/admin2/anaconda3/bin/pip3", line 11, in <module>
    sys.exit(main())
TypeError: 'module' object is not callable


In [2]:
#from adbnx_adapter import ArangoDB_Networkx_Adapter

import oasis

# Create a Temporary ArangoDB Instance

In [3]:
%%capture
!git clone https://github.com/joerg84/Graph_Powered_ML_Workshop.git
!rsync -av Graph_Powered_ML_Workshop/ ./ --exclude=.git

*Note: You can also use the above link and login data to explore the ArangoDB UI.*

# The Fraud Detection Dataset

In [4]:
import csv
import json
import requests
import sys
import oasis


from pyArango.connection import *
from pyArango.collection import Collection, Edges, Field
from pyArango.graph import Graph, EdgeDefinition
from pyArango.collection import BulkOperation as BulkOperation

In [5]:
# Retrieve tmp credentials from ArangoDB Tutorial Service
con = oasis.getTempCredentials()

print()
print("https://{}:{}".format(con["hostname"], con["port"]))
print("Username: " + con["username"])
print("Password: " + con["password"])
print("Database: " + con["dbName"])

Requesting new temp credentials.
Temp database ready to use.

https://5904e8d8a65f.arangodb.cloud:8529
Username: TUTk2ykuy4dgrpafx6m0wrk8d
Password: TUTmzj9wvl6foy8t7nmo7pxj
Database: TUTol8ui4zxzp9bapncps99rd


In [6]:
con

{'dbName': 'TUTol8ui4zxzp9bapncps99rd',
 'username': 'TUTk2ykuy4dgrpafx6m0wrk8d',
 'password': 'TUTmzj9wvl6foy8t7nmo7pxj',
 'hostname': '5904e8d8a65f.arangodb.cloud',
 'port': 8529}

In [7]:
!./tools/arangorestore -c none --server.endpoint http+ssl://{con["hostname"]}:{con["port"]} --server.username {con["username"]} --server.database {con["dbName"]} --server.password {con["password"]} --default-replication-factor 3  --input-directory "data/fraud_dump"

2020-05-06T10:11:41Z [18255] INFO [05c30] {restore} Connected to ArangoDB 'http+ssl://5904e8d8a65f.arangodb.cloud:8529'
2020-05-06T10:11:43Z [18255] INFO [abeb4] {restore} Database name in source dump is 'fraud-detection'
2020-05-06T10:11:43Z [18255] INFO [9b414] {restore} # Re-creating document collection 'account'...
2020-05-06T10:11:44Z [18255] INFO [9b414] {restore} # Re-creating document collection 'bank'...
2020-05-06T10:11:46Z [18255] INFO [9b414] {restore} # Re-creating document collection 'branch'...
2020-05-06T10:11:47Z [18255] INFO [9b414] {restore} # Re-creating document collection 'Class'...
2020-05-06T10:11:49Z [18255] INFO [9b414] {restore} # Re-creating document collection 'customer'...
2020-05-06T10:11:51Z [18255] INFO [9b414] {restore} # Re-creating edge collection 'accountHolder'...
2020-05-06T10:11:53Z [18255] INFO [9b414] {restore} # Re-creating edge collection 'Relationship'...
2020-05-06T10:11:55Z [18255] INFO [9b414] {restore} # Re-creating edge collection 'tran

TODO: arangorestore for the import of the imdb Fraud Detection data set

# Connect ArangoDB and NetworkX 

In [8]:
# Create a ArangoDB_Networkx_Adapter against the temp ArangoDB instance
from adbnx_adapter.adbnx_adapter.arangoDB_networkx_arango_adapter import ArangoDB_Networkx_Adapter

ma = ArangoDB_Networkx_Adapter(conn = con)
    

In [11]:
import sys
stdout = sys.stdout
sys.setdefaultencoding('utf-8')
sys.stdout = stdout

AttributeError: module 'sys' has no attribute 'setdefaultencoding'

In [12]:
fraud_detection_attributes = { 'vertexCollections': {'account': {'Balance', 'account_type', 'customer_id', 'rank'},\
       'bank': {'Country', 'Id', 'bank_id', 'bank_name'},\
       'branch':{'City', 'Country', 'Id', 'bank_id', 'branch_id', 'branch_name'},\
       'Class':{'concrete', 'label', 'name'},\
       'customer': {'Name', 'Sex', 'Ssn', 'rank'}},\
                              'edgeCollections' : {'accountHolder': {'_from', '_to'},\
       'Relationship': {'_from', '_to', 'label', 'name', 'relationshipType'},\
       'transaction': {'_from', '_to'}}}

g = ma.create_networkx_graph(graph_name = 'FraudDetection', graph_attributes = fraud_detection_attributes)
#g = ma.create_networkx_graph(graph_attributes = fraud_detection_attributes)

Query for vertex:
FOR doc in account RETURN { account_type:doc.account_type,rank:doc.rank,Balance:doc.Balance,customer_id:doc.customer_id,_id: doc._id}


MissingSchema: Invalid URL '5904e8d8a65f.arangodb.cloud/_db/TUTol8ui4zxzp9bapncps99rd/_api/cursor': No schema supplied. Perhaps you meant http://5904e8d8a65f.arangodb.cloud/_db/TUTol8ui4zxzp9bapncps99rd/_api/cursor?

In [ ]:
g.nodes()

In [ ]:
g.edges()